Author: Luciana Nieto 

Date: November 14th 2022


##  Libraries

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
import os
import glob

import pandas as pd
import numpy as np
import pandas as pandasForSortingCSV

In [6]:
glob.os.chdir('//content/drive/My Drive/Corteva_test/DataSciTest')
print(glob.os.getcwd())

print(glob.os.listdir())

#level up:
#glob.os.chdir('..DataSciTest/')

/content/drive/My Drive/Corteva_test/DataSciTest
['yld_data', 'wx_data', 'answers']


## Problem 1
Write a software program to calculate for each weather data file the number of days in which the maximum temperature and minimum temperature data are present but the precipitation data is missing. The program should create one line of output for each of the weather data files. Each line of output should have the filename (e.g. USC00339312.txt) and the number of days you have identified. Separate the columns by a tab. Write the output in ascending order by filename and store it in the file MissingPrcpData.out in the answers folder.


In [84]:
folder= '/content/drive/My Drive/Corteva_test/DataSciTest/wx_data'

def get_files(folder):
    import os
    os.chdir(folder)
    files = os.listdir()
    files = [x for x in files if x.endswith(".txt")]
    return files 
files = get_files(folder) 

len(files)

167

In [8]:
number = []
name =[]

for file in files: 
  data = pd.read_csv(file, index_col = False,sep="\t", header=None)
  data.replace(to_replace=-9999, value=np.nan, inplace=True)
  lit = ((data.iloc[:, 3].isnull()) & (data.iloc[:, 2].notna())).sum()
  number.append(lit)
  path = os.path.basename(file)
  name.append(path)
  #print(path)


In [10]:
t = pd.DataFrame(list(zip(name, number)))

import pandas as pandasForSortingCSV

t.sort_values(t.columns[0],
                    axis=0,
                    ascending=[True],
                    inplace=True)



In [119]:
t.to_csv(r'/content/drive/My Drive/Corteva_test/DataSciTest/answers/MissingPrcpData.out', header=None, index=None, sep='\t', mode='a')

## Problem 2
Write a software program to calculate the following values for every year for every station, ignoring missing data:

	* Average maximum temperature (in degrees Celsius)
	* Average minimum temperature (in degrees Celsius)
	* Total accumulated precipitation (in centimeters)

The program should create one line of output for each of the weather stations. Each line of output should have the following information separated by tabs: the filename, the year, the average maximum temperature to 2 decimal places, the average minimum temperature to 2 decimal places, and the total accumulated precipitation to 2 decimal places. If a value cannot be calculated assign a value of -9999.00 when writing your output.  Store your results in the file YearlyAverages.out in the answers folder, sorted in ascending order by filename.

In [421]:
pd.options.display.float_format = '{:.2f}'.format
tx =[]
tn=[]
pp =[]
year =[]
path = []
final = []
for file in files: 
  data = pd.read_csv(file, index_col = False,sep="\t", header=None)
  data.iloc[:, 1:4]=data.iloc[:, 1:4]/10
  data.iloc[:,0] = pd.to_datetime(data.iloc[:,0], format = '%Y%m%d')
  data.loc[:,0]= data.loc[:,0].astype(str).str[0:4]
  data.replace(to_replace=-999.9, value=np.nan, inplace=True)
  data.columns = [ 'year','tmx','tmn','pp' ]
  df_subset0= data.loc[:,['year']]
  df_subset1= data.loc[:,['year', 'tmx']]
  df_subset2= data.loc[:,['year', 'tmn']]
  df_subset3= data.loc[:,['year', 'pp']]
  d1= df_subset1.groupby(['year'],as_index=False).mean()
  d1 = d1.drop('year', axis=1)
  d2 = df_subset2.groupby(['year'],as_index=False).mean()
  d2 = d2.drop('year', axis=1)
  d3 = df_subset3.groupby(['year'],as_index=False).sum()
  d3 = d3.drop('year', axis=1)
  d4 = df_subset0.groupby(['year'], as_index=False).mean()
  d1.fillna(-9999.00, inplace=True)
  d2.fillna(-9999.00, inplace=True)
  d3.fillna(-9999.00, inplace=True)
  path1 = os.path.basename(file)
  path.append(path1)
  tx.append(d1)
  tn.append(d2)
  pp.append(d3)
  year.append(d4)


In [422]:
t4 = pd.DataFrame(list(zip(path, year, tx, tn, pp)))
t4.sort_values(t4.columns[0],
                    axis=0,
                    ascending=[True],
                    inplace=True)

#t4.to_csv(r'/content/drive/My Drive/Corteva_test/DataSciTest/answers/YearlyAverages.out', header=None, index=None, sep='\t', mode='a')

## Problem 3

Write a software program that tabulates how often each year from 1985-2014 had the highest average maximum temperature, highest average minimum temperature, and highest total accumulated precipitation from the set of weather stations. Use the results of Problem 2 as input to your program. The resulting output should indicate how frequently a given year had the highest values for the 167 weather stations present in wx_data.

Output should be written to the file YearHistogram.out in the answers folder. The file should contain 4 columns: the year, the count of how many weather stations had that year as the highest annual average maximum temperature, a count of how many weather stations had that year as the highest annual average minimum temperature, and a count of how many weather stations had that year as the highest total annual precipitation. Separate each column by tabs and sort the output in ascending order by year. In addition, create a histogram of the output and save the result as YearHistogram.png.

In [185]:
ppList =[]
txList =[]
tnList =[]
for file in files: 
  path1 = os.path.basename(file)
  data = pd.read_csv(file, index_col = False,sep="\t", header=None)
  data.iloc[:, 1:4]=data.iloc[:, 1:4]/10
  data.iloc[:,0] = pd.to_datetime(data.iloc[:,0], format = '%Y%m%d')
  data.loc[:,0]= data.loc[:,0].astype(str).str[0:4]
  data.columns = ['year','tmx','tmn','pp']
  data.replace(to_replace=-999.9, value=np.nan, inplace=True)
  data['filename']=path1
  #data.fillna(-9999.00, inplace=True)
  dk = data.groupby('year').aggregate({'tmx': 'mean',
                                    'tmn': 'mean',
                                     'pp': 'sum'})
  txdk = (dk.sort_values(by='tmx', ascending = False, axis=0)).drop(['tmn','pp'],axis=1)
  txdk['year'] = txdk.index
  txdk = txdk.reset_index(drop=True)
  txdk=txdk.T
  txdk = txdk.reset_index(drop=True)
  txdk = txdk.T.iloc[:1,]
  tndk = (dk.sort_values(by='tmn', ascending = False, axis=0)).drop(['tmx', 'pp'], axis=1)
  tndk['year'] = tndk.index
  tndk = tndk.reset_index(drop=True)
  tndk=tndk.T
  tndk = tndk.reset_index(drop=True)
  tndk = tndk.T.iloc[:1,]
  ppdk = (dk.sort_values(by='pp', ascending = False, axis=0)).drop(['tmx','tmn'], axis=1)
  ppdk['year'] = ppdk.index
  ppdk = ppdk.reset_index(drop=True)
  ppdk=ppdk.T
  ppdk = ppdk.reset_index(drop=True)
  ppdk = (ppdk.T).iloc[:1,]
  txList.append(txdk)
  tnList.append(tndk)
  ppList.append(ppdk)


  #print(data)

In [187]:
  txdk = (dk.sort_values(by='tmx', ascending = False, axis=0)).drop(['tmn','pp'],axis=1)
  txdk['year'] = txdk.index
  txdk = txdk.reset_index(drop=True)
  th=txdk.T
  th = th.reset_index(drop=True)
  th = th.T.loc[:0,]
  th

,0,1
0,20.07,2012


## Problem 4

Use the annual average maximum temperature, annual average minimum temperature, and total annual precipitation results from Problem 2 and calculate the Pearson correlation between these variables and the grain yield data stored in US_corn_grain_yield.txt. Output the correlations for every station. Write your output to the file Correlations.out in the answers folder. The columns of the file should be separated by tabs and will contain the filename of the weather station (e.g. USC00339312.txt) and the three correlation values to 2 decimal places. Sort the output in ascending order by filename.

In [315]:
folder= '/content/drive/My Drive/Corteva_test/'

def get_files(folder):
    import os
    os.chdir(folder)
    files = os.listdir()
    files = [x for x in files if x.endswith(".csv")]
    return files 

filescv = get_files(folder) 
len(filescv)

167

In [356]:
df_full.drop(columns=df_full.columns[0], axis=1, inplace=True)

In [357]:
df_full

,tmx,tmn,pp,year,path
0,18.72,7.17,1169.30,1985,USC00125237.txt
1,19.51,8.17,985.10,1986,USC00125237.txt
2,20.24,7.91,837.50,1987,USC00125237.txt
3,19.67,6.29,1069.10,1988,USC00125237.txt
4,17.70,6.53,1413.00,1989,USC00125237.txt
...,...,...,...,...,...
25,16.17,4.33,1034.00,2010,USC00339312.txt
26,16.31,5.05,1360.70,2011,USC00339312.txt
27,17.80,5.17,910.80,2012,USC00339312.txt
28,15.47,4.02,1099.00,2013,USC00339312.txt


In [358]:
df_full.to_csv(r'/content/drive/My Drive/Corteva_test/df_full.csv', header=None, index=None)

In [360]:
df_full = pd.read_csv('/content/drive/MyDrive/Corteva_test/df_full.csv')

In [418]:
corr_file = pd.DataFrame(filescv, columns=['stations'])
corr_file = corr_file["stations"].str.split(".csv", n = 1, expand = True)

corr_file.drop(columns=corr_file.columns[1], axis=1, inplace=True)
corr_file.columns = ['station_file']

In [419]:
tx= df_full.groupby(df_full.station).apply(lambda g: np.corrcoef(g.tx.values, g['grain'].values)[0][1]).sort_values()
tx_array= tx.to_numpy()

In [420]:
tn=df_full.groupby(df_full.station).apply(lambda g: np.corrcoef(g.tn.values, g['grain'].values)[0][1]).sort_values()
tn_array= tn.to_numpy()

In [421]:
pp = df_full.groupby(df_full.station).apply(lambda g: np.corrcoef(g.pp.values, g['grain'].values)[0][1]).sort_values()
pp_array= pp.to_numpy()

In [422]:
corr_file['tx_corr'] = tx_array.tolist()
corr_file['tn_corr'] = tn_array.tolist()
corr_file['pp_corr'] = pp_array.tolist()

In [424]:
corr_file_sorted = corr_file.sort_values('station_file' )

corr_file_sorted

,station_file,tx_corr,tn_corr,pp_corr
61,USC00110072.txt,-0.20,-0.17,0.09
95,USC00110187.txt,-0.09,-0.01,0.20
102,USC00110338.txt,-0.06,0.01,0.22
110,USC00111280.txt,-0.02,0.06,0.24
85,USC00111436.txt,-0.10,-0.07,0.17
...,...,...,...,...
75,USC00338552.txt,-0.15,-0.10,0.12
9,USC00338769.txt,-0.41,-0.46,-0.18
141,USC00338822.txt,0.11,0.27,0.32
19,USC00338830.txt,-0.38,-0.37,-0.10


In [425]:
#file output
corr_file_sorted.to_csv(r'/content/drive/My Drive/Corteva_test/DataSciTest/answers/Correlations.out', header=None, index=None, sep='\t', mode='a')